In [ ]:
import numpy as np
import pandas as pd
from gtfparse import read_gtf
import pysam as ps
from Bio.Seq import Seq

In [ ]:
# which reference transcript
hgnc = pd.read_csv('reference_transcripts.txt', sep='\t')
hgnc = hgnc.drop_duplicates(subset=['Approved symbol'])
replaceDict = {'NM_003220':'NM_001042425','NM_004241':'NM_032776','NM_022976':'NM_001144914'}
for _ in replaceDict:
    i = hgnc[hgnc['RefSeq accession']==_].index.values[0]
    hgnc.at[i,'RefSeq accession'] = replaceDict[_]

# whole hg19 (GRCh37) reference sequence
ref_seq = ps.FastaFile('reference_GRCh37.fa.gz')

# table with variants and positions
variants = pd.read_excel('variants_positions.xlsx')

variants = pd.merge(variants, hgnc[['Approved symbol','RefSeq accession']], left_on='HUGO_Symbol', 
                    right_on='Approved symbol', how='left')
variants = variants.drop(columns='Approved symbol')
variants[['CHR', 'POS', 'REF', 'ALT', 'nr']] = variants['shortcut'].str.split('_', expand=True)

In [ ]:
# GTF file with sequence information
gtf_df = read_gtf('GRCh37_sequence_information.gtf')
# only those reference transcripts within variant file
gtf_df['transcript_id'] = gtf_df['transcript_id'].str.split('.',expand=True)[0]
gtf_df = gtf_df[gtf_df['transcript_id'].isin(set(variants['RefSeq accession']))]
gtf_df=gtf_df[gtf_df['feature']=='CDS']

In [ ]:
def search_col(df, inp):
    return [i for i in df.columns if inp in i.lower()]

def rev_comp(seq):
    return str(Seq(seq).reverse_complement())

In [ ]:
# new cDNA/protein sequence with indels and variants
def cDNA(ind):
    cdna = ''
    ind_df = gtf_df[gtf_df['transcript_id']==variants.iloc[ind]['RefSeq accession']]
    kind = variants.iloc[ind]['kind']
    chrom = str(variants.iloc[ind]['#CHROM'])
    var_pos = int(variants.iloc[ind]['POS'])
    ref_len = len(variants.iloc[ind]['REF'])-1
    pos_last = int(variants.iloc[ind]['pos_last'])
    pos_next = int(variants.iloc[ind]['pos_next'])
    ori = list(ind_df['strand'])[0]
    aa_pos = 0
    hgsv_pos = 0
    
    # exclude duplicated sequences
    new_pos = 0
    
    # for new HGVSg in VCF
    ref_n = ''
    alt_n = ''
    
    for s,e in zip(ind_df['start'], ind_df['end']):
        # for insertion (new pos (start/end of new exon) in intron)
        if new_pos == 0:
            if pos_last == e:
                e = pos_next-1
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_last-1, pos_last)
                alt_n = ref_seq.fetch(chrom, pos_last-1, pos_next-1)
                hgsv_pos = pos_last
            elif pos_next == s:
                s = pos_last+1
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_next-2, pos_next-1)
                alt_n = ref_n + ref_seq.fetch(chrom, pos_last, pos_next-1)
                hgsv_pos = pos_next-1
            # for deletion (new pos (start/end of new exon) in exon)
            elif pos_next-1 == e:
                e = pos_last
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_last-1, pos_next-1)
                alt_n = ref_seq.fetch(chrom, pos_last-1, pos_last)
                hgsv_pos = pos_last
            elif pos_last+1 == s:
                s = pos_next
                new_pos +=1
                ref_n = ref_seq.fetch(chrom, pos_last-1, pos_next-1)
                alt_n = ref_seq.fetch(chrom, pos_last-1, pos_last)
                hgsv_pos = pos_last
            else:
                pass
        else:
            pass
        
        # e.g. in exon skipping events
        if s+1 > e:
            cdna += ''
        
        # if original variant is in new cDNA
        elif s <= var_pos and e >= var_pos:
            # for deletions starting in exon
            if ref_len>0:
                if ori == '+':
                    if var_pos+ref_len < e:
                        cdna += ref_seq.fetch(chrom, s-1, var_pos)
                        cdna += ref_seq.fetch(chrom, var_pos+ref_len, e)
                    else: # var_pos+ref_len >= e
                        cdna += ref_seq.fetch(chrom, s-1, var_pos)
                elif ori == '-':
                    if var_pos+ref_len < e:
                        cdna += rev_comp(ref_seq.fetch(chrom, var_pos+ref_len, e))
                        cdna += rev_comp(ref_seq.fetch(chrom, s-1, var_pos))
                    else: # var_pos+ref_len >= e
                        cdna += rev_comp(ref_seq.fetch(chrom, s-1, var_pos))
                else:
                    pass

            # for insertions, substitutions
            else:
                if ori == '+':
                    cdna += ref_seq.fetch(chrom, s-1, var_pos-1)
                    cdna += variants.iloc[ind]['ALT']
                    cdna += ref_seq.fetch(chrom, var_pos, e)
                elif ori == '-':
                    cdna += rev_comp(ref_seq.fetch(chrom, var_pos, e))
                    cdna += rev_comp(variants.iloc[ind]['ALT'])
                    cdna += rev_comp(ref_seq.fetch(chrom, s-1, var_pos-1))
                else:
                    pass
    
        # for deletions starting in intron
        elif ref_len>0 and s <= var_pos+ref_len and e >= var_pos+ref_len:
            if ori == '+':
                cdna += ref_seq.fetch(chrom, var_pos+ref_len, e)
            elif ori == '-':
                cdna += rev_comp(ref_seq.fetch(chrom, var_pos+ref_len, e))
            else:
                pass
        
        elif ori == '+':
            cdna += ref_seq.fetch(chrom, s-1, e)
        elif ori == '-':
            cdna += rev_comp(ref_seq.fetch(chrom, s-1, e))
        else:
            pass
    return [cdna, hgsv_pos, ref_n, alt_n]

In [ ]:
# for reference cDNA/protein sequence for comparison
def ref_cdna(ind):
    cdna = ''
    chrom = str(variants.iloc[ind]['#CHROM'])
    ind_df = gtf_df[gtf_df['transcript_id']==variants.iloc[ind]['RefSeq accession']]
    ori = list(ind_df['strand'])[0]
    for s,e in zip(ind_df['start'], ind_df['end']):
        if ori == '+':
            cdna += ref_seq.fetch(chrom, s-1, e)
        elif ori == '-':
            cdna += rev_comp(ref_seq.fetch(chrom, s-1, e))
        else:
            pass
    return cdna

# position of change in nucleic sequence
def nuc_pos(ind):
    alt_nuc = variants.iloc[ind]['cDNA']
    ref_nuc = variants.iloc[ind]['ref_cDNA']
    ct = 0
    nuc_pos = 0
    for a,r in zip(alt_nuc, ref_nuc[:len(alt_nuc)]):
        ct+=1
        if a != r:
            nuc_pos = ct
            break
        else:
            continue
    if nuc_pos == 0 and len(alt_nuc)<len(ref_nuc):
        nuc_pos = len(alt_nuc)+1
    elif nuc_pos == 0 and len(alt_nuc)==len(ref_nuc):
        nuc_pos = 'no_change'
    return nuc_pos

# position of change in aa sequence
def prot_pos(ind):
    alt_prot = variants.iloc[ind]['termin_prot']
    ref_prot = variants.iloc[ind]['ref_protein_seq']
    ct = 0
    aa_pos = 0
    for a,r in zip(alt_prot, ref_prot[:len(alt_prot)]):
        ct+=1
        if a != r:
            aa_pos = ct
            break
        else:
            continue
    if aa_pos == 0 and len(alt_prot)<len(ref_prot):
        aa_pos = len(alt_prot)+1
    elif aa_pos == 0 and len(alt_prot)==len(ref_prot):
        aa_pos = 'no_change'
    return aa_pos

# length of exons
def exon_len(ind):
    ind_df = gtf_df[gtf_df['transcript_id']==variants.iloc[ind]['RefSeq accession']]
    exon_lst = []
    bott = 0
    for s,e,n in zip(ind_df['start'], ind_df['end'], ind_df['exon_number']):
        exon_lst.append((n, bott+1, bott+abs(s-e)+1))
        bott += abs(s-e)+1
    return exon_lst, n

# in which exon change occurs
def changed_exon(ind):
    exon = 0
    nuc_change_pos = variants.iloc[ind]['nuc_change_pos']
    if type(nuc_change_pos) == int:
        for ex,s,e in variants.iloc[ind]['exons_lengths']:
            if nuc_change_pos >= s and nuc_change_pos <= e:
                exon = ex
            else:
                pass
    return exon

In [ ]:
# find out next matching amino acids position
import re
def aas_next(ind):
    change_pos = variants.iloc[ind]['prot_change_pos']
    ct = 0
    pos_next_normal = ''
    len_indel = 0
    
    try:
        if type(change_pos)==int:
            alt1 = variants.iloc[ind]['termin_prot']
            short_alt = alt1[change_pos-1:]
            ref1 = variants.iloc[ind]['ref_protein_seq']
            short_ref = ref1[change_pos-1:]
            kind = variants.iloc[ind]['kind']

            if kind=='ins':
                if re.search(short_ref, short_alt) and len(alt1)>len(ref1):
                    len_indel = re.search(short_ref, short_alt).regs[0][0]
                    pos_next_normal = change_pos+len_indel
                elif re.search(short_ref[1:], short_alt) and len(alt1)>len(ref1):
                    len_indel = re.search(short_ref[1:], short_alt).regs[0][0]
                    pos_next_normal = change_pos+len_indel
                else:
                    pos_next_normal = 'probably frameshift'
            elif kind =='del':
                if re.search(short_alt, short_ref) and len(ref1)>len(alt1):
                    len_indel = re.search(short_alt, short_ref).regs[0][0]
                    pos_next_normal = change_pos+len_indel
                elif re.search(short_alt[1:], short_ref) and len(ref1)>len(alt1):
                    len_indel = re.search(short_alt[1:], short_ref).regs[0][0]
                    pos_next_normal = change_pos+len_indel
                else:
                    pos_next_normal = 'probably frameshift'
            else:
                pass
        else:
            pos_next_normal = 'no change'
        if abs(len(alt1)-len(ref1)) > (len_indel+1):
            pos_next_normal = 'probably frameshift'
        else:
            pass
    except:
        pos_next_normal = 'error'
    return [pos_next_normal, len_indel]

In [ ]:
# new columns in variants df with cDNA, protein sequence
variants[['cDNA', 'hgsv_pos', 'ref_n', 'alt_n']] = [cDNA(i) for i in variants.index]
variants['protein_seq'] = [str(Seq(cDNA(i)[0]).translate()) for i in variants.index]
# terminated protein sequence after first stop codon
variants['termin_prot'] = variants['protein_seq'].str.split('*', expand=True)[0]

# new columns in variants df with reference cDNA, protein sequence
variants['ref_cDNA'] = [ref_cdna(i) for i in variants.index]
variants['ref_protein_seq'] = [str(Seq(ref_cdna(i)).translate()) for i in variants.index]

# length of sequences
for i in ['cDNA', 'termin_prot', 'ref_cDNA', 'ref_protein_seq']:
    variants[i+'_len'] = variants[i].str.len()

# position of change in nucleic/aa sequence
variants['nuc_change_pos'] = [nuc_pos(i) for i in variants.index]
variants['prot_change_pos'] = [prot_pos(i) for i in variants.index]

# position of next matching aa and length of indel
variants[['aa_next_pos', 'len_indel_substit']] = [aas_next(i) for i in variants.index]

# length of exons
variants[['exons_lengths', 'max_exon']] = [exon_len(i) for i in variants.index]

# changed exon
variants['changed_exon'] = [changed_exon(i) for i in variants.index]

#### Affected protein domains

In [ ]:
with open('prot_seqs_folding.txt', 'w') as o:
    for sc, gene, alt, ref in zip(variants['shortcut'], variants['HUGO_Symbol'], 
                                  variants['termin_prot'], variants['ref_protein_seq']):
        o.write('>%s\n%s\n>%s\n%s\n'%(sc,alt,gene,ref))

In [ ]:
# import InterProScan TSVs as df-dictionary
files = os.listdir('InterProScan_directory')
ip_dfs = {}
for file in files:
    df = pd.read_csv(('InterProScan_directory/'+file), sep='\t', 
                      names=['seq', 'ID', 'prot_len', 'where_data_from', 'domain_short', 'info_domain', 'start_pos',
                             'end_pos', 'probab', 'sth', 'date', 'ipr_id', 'more_info', 'go_id', 'many'], 
                      usecols=['where_data_from', 'domain_short', 'info_domain', 'start_pos', 'end_pos',
                                'ipr_id', 'more_info', 'go_id'])
    ip_dfs[file.split('_')[1].split('.')[0].upper()] = df.sort_values(by=['start_pos', 'end_pos']
                                                                     ).reset_index(drop=True)

In [ ]:
# check, which domains are effected
def check_domains(ind):
    domains = []
    gene = variants.iloc[ind]['HUGO_Symbol']
    pos_prot_change = variants.iloc[ind]['prot_change_pos']
    if gene in ip_dfs.keys() and type(pos_prot_change) == int:
        ip_scan = ip_dfs[variants.iloc[ind]['HUGO_Symbol']]
        for s,e,dom in zip(ip_scan['start_pos'], ip_scan['end_pos'], ip_scan['domain_short']):
            if pos_prot_change >= s and pos_prot_change <= e:
                domains.append(dom)
    return domains

# check, which domains are deleted
def deleted_domains(ind):
    domains = []
    gene = variants.iloc[ind]['HUGO_Symbol']
    pos_prot_change = variants.iloc[ind]['prot_change_pos']
    termin_prot_len = variants.iloc[ind]['termin_prot_len']
    if gene in ip_dfs.keys() and type(pos_prot_change) == int:
        ip_scan = ip_dfs[variants.iloc[ind]['HUGO_Symbol']]
        for s,e,dom in zip(ip_scan['start_pos'], ip_scan['end_pos'], ip_scan['domain_short']):
            if termin_prot_len <= s:
                domains.append(dom)
    return domains

# new columns with effected/deleted domains
variants['domains_effected'] = [check_domains(i) for i in variants.index]
variants['domains_deleted'] = [deleted_domains(i) for i in variants.index]